https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_for_knowledge_retrieval.ipynb

In [1]:
import openai
from projects.chatgpt.agents.LLMFunctionsAgent import LLMFunctionsAgent

# configure the client's secret key
with open('secret-key.txt', 'r') as file:
    secret_key = file.read().strip()
openai.api_key = secret_key

max_tokens = 4092

# automatically reload changes to imports
%load_ext autoreload
%autoreload

In [2]:
import time
def prompt(question):
    start_time_seconds = time.time()
    print(f"asking chatgpt: {question}")
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", #"text-davinci-003",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that answers questions accurately, without making up facts."},
            {"role": "user", "content": question}
        ],
    )
    # print(response)
    choices = response.choices
    choice = choices[0]
    message = choice['message']
    answer = message['content']
    print(answer)
    print(f"answer received in {time.time() - start_time_seconds} seconds.")

def stream_prompt(question):
    start_time_seconds = time.time()
    print(f"asking chatgpt: {question}")
    stream = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", #"text-davinci-003",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that answers questions accurately, without making up facts."},
            {"role": "user", "content": question}
        ],
        stream=True,
    )
    for output in stream:
        choices = output['choices']
        choice = choices[0]
        delta = choice['delta']
        if 'content' in delta:
            print(delta['content'], end='')
    print()
    print(f"answer received in {time.time() - start_time_seconds} seconds.")

In [3]:
stream_prompt("""
Who is Jason McAffee?
""")

asking chatgpt: 
Who is Jason McAffee?
I couldn't find any specific information about a person named Jason McAffee. It's possible that he is a private individual or not widely known in public sources.
answer received in 1.4250340461730957 seconds.


In [4]:
%autoreload

llm_functions_agent = LLMFunctionsAgent(openai=openai)

def inference(question):
    llm_functions_agent.inference(question)


init

---- found function with tool_data: find_financial_transactions
---- found function with tool_data: get_user_details


In [5]:
%autoreload

inference("""
Who is user Jason McAffee?
""")

------- inference ------------
sending messages: [{'role': 'system', 'content': 'You are a helpful assistant that answers questions accurately, without making up facts.'}, {'role': 'user', 'content': '\nWho is user Jason McAffee?\n'}]


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
